In [108]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.memory import ConversationBufferMemory
from langchain.chains.llm import LLMChain
from langchain.chains import ConversationChain
from langchain.schema import AIMessage, HumanMessage, SystemMessage, messages_from_dict

from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

import openai
from dotenv import load_dotenv

In [2]:
load_dotenv("../app/.env")

True

In [3]:
openai.api_key = os.environ["OPENAI_API_KEY"]
openai.organization = os.environ["OPENAI_ORG_ID"]

In [4]:
def convert_cml_to_langchain_format(message):
    if message["role"] == "assistant":
        message_cls = AIMessage
    elif message["role"] == "user":
        message_cls = HumanMessage
    else:
        message_cls = SystemMessage

    return message_cls(content=message["content"])

In [46]:
def convert_cml_messages_to_langchain_format(messages):
    return [convert_cml_to_langchain_format(message) for message in messages]

In [135]:
router_messages = [
    {
        "role": "assistant",
        "content": "Please analyze the following JavaScript expression and break it down into simpler parts, explaining the order of operations and the intermediate results at each step: `3 + 5 * 2 - 4 / 2`. Write your answer in the format of step-by-step explanations.",
    },
    {
        "role": "user",
        "content": """- 3 + 5 = 8
- 2 - 4 = -2
- 8 * -2 / 2
- 8 * -2 = -16
- -16/2 = -8""",
    },
    {
        "role": "assistant",
        "content": """Actual solution:
```
1. 5 * 2 = 10 (multiplication has higher precedence than addition and subtraction)
2. 4 / 2 = 2 (division has higher precedence than addition and subtraction)
3. 3 + 10 = 13 (addition)
4. 13 - 2 = 11 (subtraction)
```
{"Answer": 0, "feedback": "You didn't follow the correct order of operations. Remember that multiplication and division have higher precedence than addition and subtraction. Please try again, keeping the order of operations in mind."}""",
    },
]

In [136]:
router_langchain_messages = [
    convert_cml_to_langchain_format(message) for message in router_messages
]

In [137]:
chat_model = ChatOpenAI(temperature=0, model="gpt-4")

In [138]:
router_system_prompt_template = """You will be provided with a series of interactions between a student and an interviewer along with a student query. The interviewer has asked the student a particular question. The student query will be delimited with #### characters.
 
Classify each query into one of the categories below:
- Answer to the question
- Clarifying question
- Irrelevant to the question

{format_instructions}
"""

In [139]:
router_output_schema = ResponseSchema(
    name="type",
    description="either of 'answer', 'clarification', 'irrelevant'",
    type="answer | clarification | irrelevant",
)
router_output_parser = StructuredOutputParser.from_response_schemas(
    [router_output_schema]
)
router_format_instructions = router_output_parser.get_format_instructions()

In [140]:
router_system_prompt = SystemMessage(
    content=router_system_prompt_template.format(
        format_instructions=router_format_instructions
    )
)

In [141]:
router_langchain_messages = [router_system_prompt] + router_langchain_messages

In [142]:
router_memory = ConversationBufferMemory()

In [143]:
router_memory.chat_memory.messages = router_langchain_messages

In [144]:
router_user_prompt_template_message = "####{input}####"

In [145]:
# router_system_prompt_template = SystemMessagePromptTemplate.from_template(
#     template=router_system_prompt_template_message
# )
# router_user_prompt_template = HumanMessagePromptTemplate.from_template(
#     template=router_user_prompt_template_message
# )
# router_chat_prompt_template = ChatPromptTemplate.from_messages([router_system_prompt])

In [146]:
# router_chat_prompt_template.input_variables

In [147]:
router_chain = ConversationChain(
    llm=chat_model,
    memory=router_memory,
    verbose=True,
)

In [148]:
router_input = router_user_prompt_template_message.format(input="How can I improve?")

In [149]:
router_chain.run(router_input)



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: You will be provided with a series of interactions between a student and an interviewer along with a student query. The interviewer has asked the student a particular question. The student query will be delimited with #### characters.
 
Classify each query into one of the categories below:
- Answer to the question
- Clarifying question
- Irrelevant to the question

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"type": answer | clarification | irrelevant  // either of 'answer', 'clarification', 'irrelevant'
}
```

AI: Please analyze the following JavaScript expression and break 

'```json\n{\n\t"type": "clarification"\n}\n```'

In [151]:
router_output_parser.parse(router_chain.run("Give me more candies"))



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: You will be provided with a series of interactions between a student and an interviewer along with a student query. The interviewer has asked the student a particular question. The student query will be delimited with #### characters.
 
Classify each query into one of the categories below:
- Answer to the question
- Clarifying question
- Irrelevant to the question

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"type": answer | clarification | irrelevant  // either of 'answer', 'clarification', 'irrelevant'
}
```

AI: Please analyze the following JavaScript expression and break 

{'type': 'irrelevant'}

In [184]:
# answer_evaluation_messages = [
#     {
#         "role": "user",
#         "content": """,
#     }
# ]

In [185]:
# answer_evaluation_langchain_messages = [
#     convert_cml_to_langchain_format(message) for message in answer_evaluation_messages
# ]

In [220]:
answer_evaluation_system_prompt_template = """You are a helpful and encouraging interviewer.
You will be specified with a topic, a blooms level, and learning outcome along with a question that the student needs to be tested on as well as the student's response to the question. You need to provide an evaluation based on the student's response. The student's response will be delimited with #### characters.

To solve the problem, do the following
- First, work out your own solution to the problem
- Then, compare your solution to the student's solution. Don’t give any answers or hints. Assess the student on the learning outcome provided using a rating of 0 (Unsatisfactory), 1 (Satisfactory) or 2 (Proficient).
- At the end, give some actionable feedback too.

Use the following format:
Actual solution:
{{ concise steps to work out the solution and your solution here }}

{format_instructions}"""

In [221]:
answer_evaluation_answer_schema = ResponseSchema(
    name="answer",
    description="the final evaluation",
    type="integer",
)
answer_evaluation_feedback_schema = ResponseSchema(
    name="feedback",
    description="the feedback for the student",
    type="string",
)
answer_evaluation_output_parser = StructuredOutputParser.from_response_schemas(
    [answer_evaluation_answer_schema, answer_evaluation_feedback_schema]
)
answer_evaluation_format_instructions = (
    answer_evaluation_output_parser.get_format_instructions()
)

In [222]:
answer_evaluation_system_prompt = SystemMessage(
    content=answer_evaluation_system_prompt_template.format(
        format_instructions=answer_evaluation_format_instructions
    )
)

In [223]:
answer_evaluation_langchain_messages = [answer_evaluation_system_prompt]

In [224]:
answer_evaluation_langchain_messages

[SystemMessage(content='You are a helpful and encouraging interviewer.\nYou will be specified with a topic, a blooms level, and learning outcome along with a question that the student needs to be tested on as well as the student\'s response to the question. You need to provide an evaluation based on the student\'s response. The student\'s response will be delimited with #### characters.\n\nTo solve the problem, do the following\n- First, work out your own solution to the problem\n- Then, compare your solution to the student\'s solution. Don’t give any answers or hints. Assess the student on the learning outcome provided using a rating of 0 (Unsatisfactory), 1 (Satisfactory) or 2 (Proficient).\n- At the end, give some actionable feedback too.\n\nUse the following format:\nActual solution:\n{ concise steps to work out the solution and your solution here }\n\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "`

In [226]:
answer_evaluation_memory = ConversationBufferMemory()

In [227]:
answer_evaluation_memory.chat_memory.messages = answer_evaluation_langchain_messages

In [228]:
answer_evaluation_user_prompt_template_message = "####{input}####"

In [229]:
answer_evaluation_chain = ConversationChain(
    llm=chat_model,
    memory=answer_evaluation_memory,
    verbose=True,
)

In [230]:
answer_evaluation_input = answer_evaluation_user_prompt_template_message.format(
    input="""Topic: “Javascript” -> “Basics” -> “Operators”
Blooms level: “Analysing”
Learning outcome: “Analyze complex expressions containing multiple operators, and break them down into simpler parts.”
Question: Please analyze the following JavaScript expression and break it down into simpler parts, explaining the order of operations and the intermediate results at each step: `3 + 5 * 2 - 4 / 2`. Write your answer in the format of step-by-step explanations.
Student's solution:
####
- 3 + 5 = 8
- 2 - 4 = -2
- 8 * -2 / 2
- 8 * -2 = -16
- -16/2 = -8
####"""
)

In [231]:
answer_evaluation_chain.run(answer_evaluation_input)



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: You are a helpful and encouraging interviewer.
You will be specified with a topic, a blooms level, and learning outcome along with a question that the student needs to be tested on as well as the student's response to the question. You need to provide an evaluation based on the student's response. The student's response will be delimited with #### characters.

To solve the problem, do the following
- First, work out your own solution to the problem
- Then, compare your solution to the student's solution. Don’t give any answers or hints. Assess the student on the learning outcome provided using a rating of 0 (Unsatisfactory), 1 (Satisfactory) or 2 (Proficient).
- At the end, giv

'Actual solution:\n1. 5 * 2 = 10 (multiplication)\n2. 4 / 2 = 2 (division)\n3. 3 + 10 = 13 (addition)\n4. 13 - 2 = 11 (subtraction)\n\n```json\n{\n\t"answer": 0,\n\t"feedback": "Your solution does not follow the correct order of operations. Remember to follow the PEMDAS/BODMAS rule: Parentheses/Brackets, Exponents/Orders, Multiplication and Division (left-to-right), and Addition and Subtraction (left-to-right). Please try again, keeping this rule in mind."\n}\n```'

In [233]:
answer_evaluation_output_parser.parse(
    'Actual solution:\n1. 5 * 2 = 10 (multiplication)\n2. 4 / 2 = 2 (division)\n3. 3 + 10 = 13 (addition)\n4. 13 - 2 = 11 (subtraction)\n\n```json\n{\n\t"answer": 0,\n\t"feedback": "Your solution does not follow the correct order of operations. Remember to follow the PEMDAS/BODMAS rule: Parentheses/Brackets, Exponents/Orders, Multiplication and Division (left-to-right), and Addition and Subtraction (left-to-right). Please try again, keeping this rule in mind."\n}\n```'
)

{'answer': 0,
 'feedback': 'Your solution does not follow the correct order of operations. Remember to follow the PEMDAS/BODMAS rule: Parentheses/Brackets, Exponents/Orders, Multiplication and Division (left-to-right), and Addition and Subtraction (left-to-right). Please try again, keeping this rule in mind.'}

In [154]:
clarify_question_messages = [
    {
        "role": "user",
        "content": """Topic: “Javascript” -> “Basics” -> “Operators”
Blooms level: “Analysing”
Learning outcome: “Analyze complex expressions containing multiple operators, and break them down into simpler parts.”
Question: Please analyze the following JavaScript expression and break it down into simpler parts, explaining the order of operations and the intermediate results at each step: `3 + 5 * 2 - 4 / 2`. Write your answer in the format of step-by-step explanations.
Student's solution:
- 3 + 5 = 8
- 2 - 4 = -2
- 8 * -2 / 2
- 8 * -2 = -16
- -16/2 = -8""",
    },
    {
        "role": "assistant",
        "content": """Actual solution:
```
1. 5 * 2 = 10 (multiplication)
2. 4 / 2 = 2 (division)
3. 3 + 10 = 13 (addition)
4. 13 - 2 = 11 (subtraction)
```
{"Answer": 0, "feedback": "You have not followed the order of operations correctly. Remember to follow the order: multiplication and division (from left to right), then addition and subtraction (from left to right). Reanalyze the given expression by following the correct order of operations, and try solving it again."}""",
    },
    {"role": "user", "content": "Where did I go wrong?"},
    {
        "role": "assistant",
        "content": """You went wrong by not following the correct order of operations. In JavaScript, as well as in most programming languages, the order of operations is:

1. Parentheses
2. Multiplication and division (from left to right)
3. Addition and subtraction (from left to right)

So, for the expression `3 + 5 * 2 - 4 / 2`, you should perform the multiplication and division first, then addition and subtraction. Make sure you follow the proper order and try to reanalyze the given expression step-by-step.""",
    },
]

In [155]:
clarify_question_langchain_messages = convert_cml_messages_to_langchain_format(
    clarify_question_messages
)

In [156]:
clarify_question_system_prompt_message = """You are a helpful and encouraging interviewer.
You will be specified with a topic, a blooms level, and learning outcome along with a question that the student needs to be tested on along with a series of interactions between a student and you.

The student has asked a clarifying question that you need to answer. The student's response will be delimited with #### characters

Important:
- Make sure to not give hints or answer the question.
- If the student asks for the answer, refrain from answering."""

In [157]:
clarify_question_system_prompt = SystemMessage(
    content=clarify_question_system_prompt_message
)

In [158]:
clarify_question_langchain_messages = [
    clarify_question_system_prompt
] + clarify_question_langchain_messages

In [160]:
clarify_question_user_prompt_template_message = "####{input}####"

In [161]:
# router_system_prompt_template = SystemMessagePromptTemplate.from_template(
#     template=router_system_prompt_template_message
# )
# clarify_question_user_prompt_template = HumanMessagePromptTemplate.from_template(
#     template=clarify_question_user_prompt_template_message
# )
# clarify_question_chat_prompt_template = ChatPromptTemplate.from_messages(
#     [clarify_question_system_prompt, clarify_question_user_prompt_template]
# )

In [162]:
clarify_question_memory = ConversationBufferMemory()

In [163]:
clarify_question_memory.chat_memory.messages = clarify_question_langchain_messages

In [165]:
clarify_question_chain = ConversationChain(
    llm=chat_model,
    memory=clarify_question_memory,
    verbose=True,
)

In [166]:
clarify_question_input = clarify_question_user_prompt_template_message.format(
    input="How can I learn about this? "
)

In [167]:
clarify_question_chain.run(clarify_question_input)



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: You are a helpful and encouraging interviewer.
You will be specified with a topic, a blooms level, and learning outcome along with a question that the student needs to be tested on along with a series of interactions between a student and you.

The student has asked a clarifying question that you need to answer. The student's response will be delimited with #### characters

Important:
- Make sure to not give hints or answer the question.
- If the student asks for the answer, refrain from answering.
Human: Topic: “Javascript” -> “Basics” -> “Operators”
Blooms level: “Analysing”
Learning outcome: “Analyze complex expressions containing multiple operators, and break them down into

"To learn about the order of operations in JavaScript and programming in general, you can follow these steps:\n\n1. Read online tutorials and articles: There are many resources available online that explain the order of operations in programming languages, including JavaScript. Some popular websites include Mozilla Developer Network (MDN), W3Schools, and GeeksforGeeks.\n\n2. Watch video tutorials: YouTube and other video platforms have numerous tutorials on JavaScript and programming concepts, including the order of operations. Look for videos from reputable sources or instructors with good reviews.\n\n3. Practice with coding exercises: Websites like HackerRank, LeetCode, and Codewars offer coding challenges and exercises that can help you practice and reinforce your understanding of the order of operations.\n\n4. Join online forums and communities: Engage with other learners and experienced developers on forums like Stack Overflow, Reddit, or Discord channels dedicated to JavaScript a

In [168]:
print(
    "To learn about the order of operations in JavaScript and programming in general, you can follow these steps:\n\n1. Read online tutorials and articles: There are many resources available online that explain the order of operations in programming languages, including JavaScript. Some popular websites include Mozilla Developer Network (MDN), W3Schools, and GeeksforGeeks.\n\n2. Watch video tutorials: YouTube and other video platforms have numerous tutorials on JavaScript and programming concepts, including the order of operations. Look for videos from reputable sources or instructors with good reviews.\n\n3. Practice with coding exercises: Websites like HackerRank, LeetCode, and Codewars offer coding challenges and exercises that can help you practice and reinforce your understanding of the order of operations.\n\n4. Join online forums and communities: Engage with other learners and experienced developers on forums like Stack Overflow, Reddit, or Discord channels dedicated to JavaScript and programming. You can ask questions, share your knowledge, and learn from others' experiences.\n\n5. Take a course or attend a workshop: Many online platforms like Coursera, Udemy, and edX offer courses on JavaScript and programming fundamentals. You can also attend workshops or meetups in your local area to learn from experienced instructors and network with other learners.\n\nRemember, practice is key to mastering any concept, so keep working on exercises and real-world projects to solidify your understanding of the order of operations in JavaScript."
)

To learn about the order of operations in JavaScript and programming in general, you can follow these steps:

1. Read online tutorials and articles: There are many resources available online that explain the order of operations in programming languages, including JavaScript. Some popular websites include Mozilla Developer Network (MDN), W3Schools, and GeeksforGeeks.

2. Watch video tutorials: YouTube and other video platforms have numerous tutorials on JavaScript and programming concepts, including the order of operations. Look for videos from reputable sources or instructors with good reviews.

3. Practice with coding exercises: Websites like HackerRank, LeetCode, and Codewars offer coding challenges and exercises that can help you practice and reinforce your understanding of the order of operations.

4. Join online forums and communities: Engage with other learners and experienced developers on forums like Stack Overflow, Reddit, or Discord channels dedicated to JavaScript and progra